# Types of caching in snowflake:

## Cloud Service layer:

* Result Cache
    * Reuse the results displayed if:
        * table data not changed
        * Micro-Partitions not changed
        * No UDF / External functions
        * Sufficinet priviledges
        * Can be disabled with "USE_CACHED_RESULT" PARAMETER
        * If not reused, purged after 24 hrs.
* Metadata Cache
    * Count rows, distinct values, min/max.
    * No use of virtual warehouse
    * Used for Describe / System defined functions
    * VPS Edition has dedicated meta store


## Query Processing Layer:
* Data Cache
    * Stored under Local SSD of the virtual warehouse
    * Operations ran in the individual virtual warehouse
    * Dropped when warehouse is suspended or resized.

    

In [ ]:
USE SCHEMA SNOWFLAKE_SAMPLE_DATA.TPCH_SF10;
USE ROLE ACCOUNTADMIN;

## Result Cache

In [ ]:
-- Running the first, no cache will be used.
SELECT * FROM ORDERS;

-- Run again to check the difference


### last_query_id() / GET_QUERY_OPERATOR_STATS()

## last_query_id() 
when running via Notebook, we need to look for the 2nd from last query, because the last one will be a query do format the SQL result.

* last_query_id(-1) = same as last_query_id() = Gets the latest query.
* last_query_id(-2) = 2nd from last and so on.

###  GET_QUERY_OPERATOR_STATS()
Returns statistics about individual query operators within a query that has completed.
You can run this function for any completed query that was executed in the past 14 days.

https://docs.snowflake.com/en/sql-reference/functions/get_query_operator_stats


In [ ]:

select operator_type, * 
from table(get_query_operator_stats(last_query_id(-2)));


In [ ]:
ALTER WAREHOUSE COMPUTE_WH SET AUTO_RESUME = FALSE;
ALTER WAREHOUSE COMPUTE_WH SUSPEND;
ALTER WAREHOUSE COMPUTE_WH RESUME;
-- When Result Cache is not used

In [ ]:
-- Running the first, no cache will be used.
SELECT * FROM ORDERS;



 ***result cache*** it is still reused, even if the virtual warehouse is resized.
 The reason for that is that this type of cache resides in the cloud service layer.

In [ ]:

select operator_type, * 
from table(get_query_operator_stats(last_query_id(-2)));


In [ ]:
SELECT * FROM ORDERS LIMIT 4;

### any change in the query will not use the cache.

In [ ]:

select operator_type, * 
from table(get_query_operator_stats(last_query_id(-2)));


In [ ]:
ALTER SESSION SET USE_CACHED_RESULT = FALSE; --> will disable the cache
SELECT * FROM ORDERS ;

select operator_type, * from table(get_query_operator_stats(last_query_id(-1)));


## Metadata Cache

In [ ]:
-- 1. Statistics about table objects
SELECT COUNT(*) FROM ORDERS;
select operator_type, * from table(get_query_operator_stats(last_query_id(-1)));



In [ ]:
SHOW ROLES;
select operator_type, * from table(get_query_operator_stats(last_query_id(-1)));



In [ ]:
DESC TABLE ORDERS;

In [ ]:
DESC TABLE ORDERS;
select operator_type, * from table(get_query_operator_stats(last_query_id(-1)));

-- DESC / USE does not even generate query profile, so nothing will be shown here